<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Breakout_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2023-07-28 16:16:06


In [2]:
!pip install yfinance

In [3]:
import warnings
warnings.filterwarnings("ignore")
import yfinance as yfin
#import talib
yfin.pdr_override()
#import pandas_ta as ta
import pandas as pd

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/myProspectsScrips.csv')

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

In [7]:
def get_breakout(stock_symbol):

  stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close'])]
  stock_df['Stock'] = stock_symbol
  stock_df['Open'] = round(stock_df['Open'],0)
  stock_df['Close'] = round(stock_df['Close'],0)
  stock_df['PrevClose'] = round(stock_df['Close'].shift(1),0)

  stock_df = stock_df.reset_index()
  prev_max = max(stock_df[(stock_df['Date'] < '2023-01-01')]['Close'])
  stock_df['PrevMax'] = prev_max
  tmp_df = stock_df[(stock_df['Close'] > prev_max)]
  tmp_df['DateGroup'] = tmp_df['Date'].diff().dt.days.ne(1).cumsum()

  tmp_df['Date'] = tmp_df['Date'].dt.strftime('%Y-%m-%d')
  tmp_df = tmp_df[tmp_df['Close'] > tmp_df['Open']]
  tmp_df = tmp_df[tmp_df['Open'] > tmp_df['PrevClose']]
  g = tmp_df.groupby('DateGroup')['Date'].agg(['min', 'max', 'count']).reset_index()

  df_res = tmp_df.merge(g, how='left')
  df_res = df_res[df_res['count'] > 1].head(1)
  df_res['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_symbol.upper()]['Conviction'].values[0]
  df_res['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_symbol.upper()]['LatestQtr'].values[0]
  df_res['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_symbol.upper()]['StarStock'].values[0]
  df_res['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_symbol.upper()]['Criteria'].values[0]
  df_res['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_symbol.upper()]['Strategy'].values[0]

  return df_res

In [8]:
df_breakout = pd.DataFrame()

for stock_id in stock_ids:
  tmp_df = get_breakout(stock_id+'.NS')
  df_breakout = df_breakout.append(tmp_df)

df_breakout.sort_values(by = 'Date', ascending=False)

,Date,Open,Close,Stock,PrevClose,PrevMax,DateGroup,min,max,count,Conviction,LatestQtr,StarStock,Criteria,Strategy
0,2023-07-26,612.0,615.0,SBIN.NS,610.0,614.0,3,2023-07-26,2023-07-27,2,M,0,1.0,NaN,NaN
0,2023-07-25,5420.0,5425.0,DRREDDY.NS,5392.0,5408.0,1,2023-07-25,2023-07-28,3,H,1,1.0,NaN,NaN
3,2023-07-18,22965.0,23059.0,ABBOTINDIA.NS,22868.0,22611.0,5,2023-07-18,2023-07-20,2,M,0,1.0,V40,BTT
0,2023-07-18,2817.0,2820.0,RELIANCE.NS,2797.0,2811.0,1,2023-07-18,2023-07-19,2,H,1,1.0,V40,ATH
3,2023-07-17,269.0,270.0,TIPSINDLTD.NS,268.0,236.0,6,2023-07-17,2023-07-19,3,L,1,1.0,NaN,NaN
2,2023-07-17,328.0,334.0,CANBK.NS,327.0,321.0,4,2023-07-17,2023-07-19,2,H,1,1.0,NaN,NaN
2,2023-07-17,963.0,969.0,ICICIBANK.NS,960.0,953.0,4,2023-07-17,2023-07-20,2,H,1,1.0,V40,NaN
1,2023-07-12,7190.0,7401.0,BAJAJHLDNG.NS,7177.0,7031.0,4,2023-07-12,2023-07-14,2,L,0,0.0,V40,NaN
3,2023-07-11,410.0,411.0,UJJIVAN.NS,408.0,395.0,4,2023-07-11,2023-07-14,3,M,0,1.0,NaN,NaN
2,2023-07-11,1048.0,1068.0,SUNPHARMA.NS,1041.0,1040.0,3,2023-07-11,2023-07-12,2,H,0,1.0,NaN,NaN
